In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
george_images = [f for f in os.listdir('./george_test_task/george') if f.endswith('.jpg')]
non_george_images = [f for f in os.listdir('./george_test_task/no_george') if f.endswith('.jpg')]

In [2]:
def load_preprocess_image(image_path, target_size=(224, 224)):
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image)
    image = image / 255.
    return image

In [3]:
# Load and preprocess images
george_images_data = np.array([load_preprocess_image(os.path.join('./george_test_task/george', img)) for img in george_images])
non_george_images_data = np.array([load_preprocess_image(os.path.join('./george_test_task/no_george', img)) for img in non_george_images])

# Create labels
george_labels = np.ones((len(george_images), 1))
non_george_labels = np.zeros((len(non_george_images), 1))

# Combine data and labels
X = np.concatenate((george_images_data, non_george_images_data))
y = np.concatenate((george_labels, non_george_labels))

In [4]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten

# Load pre-trained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

58889256/58889256 [==============================] - 17s 0us/step


In [5]:
from sklearn.model_selection import train_test_split
# Combine data and labels
X = np.concatenate((george_images_data, non_george_images_data))
y = np.concatenate((george_labels, non_george_labels))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
143/143 [==============================] - 1498s 10s/step - loss: 0.5365 - accuracy: 0.7908 - val_loss: 0.3684 - val_accuracy: 0.8500
Epoch 2/10
143/143 [==============================] - 1609s 11s/step - loss: 0.2540 - accuracy: 0.8987 - val_loss: 0.3835 - val_accuracy: 0.8491
Epoch 3/10
143/143 [==============================] - 1455s 10s/step - loss: 0.1580 - accuracy: 0.9423 - val_loss: 0.3497 - val_accuracy: 0.8658
Epoch 4/10
143/143 [==============================] - 1230s 9s/step - loss: 0.0975 - accuracy: 0.9691 - val_loss: 0.3948 - val_accuracy: 0.8526
Epoch 5/10
143/143 [==============================] - 2619s 18s/step - loss: 0.0601 - accuracy: 0.9836 - val_loss: 0.4562 - val_accuracy: 0.8561
Epoch 6/10
143/143 [==============================] - 3250s 23s/step - loss: 0.0424 - accuracy: 0.9886 - val_loss: 0.6003 - val_accuracy: 0.8465
Epoch 7/10
143/143 [==============================] - 1224s 9s/step - loss: 0.0442 - accuracy: 0.9893 - val_loss: 0.4532 - val_accu

In [6]:
from sklearn.metrics import accuracy_score
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

# Print the test accuracy
print(f'Test accuracy: {test_accuracy}')

# Make predictions on the test data
y_pred = model.predict(X_test)

# Convert predicted probabilities to binary labels
y_pred = np.round(y_pred)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(f'Test accuracy: {accuracy}')

Test accuracy: 0.8640350699424744
36/36 [==============================] - 251s 7s/step
Test accuracy: 0.8640350877192983


In [7]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Test accuracy: {accuracy}')
print(f'Test precision: {precision}')
print(f'Test recall: {recall}')
print(f'Test F1 score: {f1}')

Test accuracy: 0.8640350877192983
Test precision: 0.8488612836438924
Test recall: 0.8333333333333334
Test F1 score: 0.841025641025641


In [8]:
def predict_image(model, image_path):
    image = load_preprocess_image(image_path)
    image = np.expand_dims(image, axis=0)
    prediction = model.predict(image)
    if prediction > 0.5:
        print ('St. George present')
    else:
        print ('St. George absent')

In [9]:
# Load an image
image_path = 'george_test_task/no_george/0a1a0665ac98ae81be4b54db17564f34.jpg'

# Make a prediction on the image
predict_image(model, image_path)

1/1 [==============================] - 0s 247ms/step
St. George absent


In [10]:
# Load an image
image_path = 'george_test_task/george/0a5f7b5996063605dd05887ef4d31855.jpg'

# Make a prediction on the image
predict_image(model, image_path)

1/1 [==============================] - 0s 279ms/step
St. George present
